**LICENCE:**
This tutorial contains adaptations of material from [Programming Language Foundations in Agda](https://plfa.github.io/) by Phil Wadler and Wen Kokke. It is licensed under Creative Commons Attribution 4.0 International.

# Curry-Howard isomorphism

\begin{align*}
    &\textrm{Logic} && &&\textrm{Computer science}\\
    \hline
    &\textrm{Theorems} &\leftrightarrow &&&
        \textrm{Types} && A \\
    &\textrm{Proofs} &\leftrightarrow &&&
        \textrm{Programs (terms)}
            && a \\
    &\textrm{Proof simplification} &\leftrightarrow &&&
        \textrm{Program execution}
            && \textrm{run } a \\
    \hline
    &\textrm{Proof development} &\leftrightarrow &&&
        \textrm{Programming}
            && \textrm{write } a \\
    &\textrm{Proof checking} &\leftrightarrow &&&
        \textrm{Type checking}
            && \textrm{given } a \textrm{ and } A, \textrm{ does } a : A?\\
    &\textrm{Validity (mathematics)} &\leftrightarrow &&&
        \textrm{Inhabitation problem}
            && \textrm{given } A, \textrm{ is there some $a : A$?}
\end{align*}

**SYNTAX:**
You can enter `→` by writing `->` and pressing TAB; pressing TAB again goes back to `->`. Similar useful combinations are:

| base form | alternate form |
|:---------:|:--------------:|
| ->  | → |
| \   | λ |
| neg | ¬ |
| top | ⊤ |
| bot | ⊥ |
| /\  | ∧ |
| \\/ | ∨ |
| <   | ⟨ |
| >   | ⟩ |
| =<> | ≡⟨⟩ |
| qed | ∎ |
| N   | ℕ |
| Pi  | Π |
| Sigma | Σ |
| forall | ∀ |
| exists | ∃ |

<!---

**PRO TRICKS:**
- You can ask Agda to provide the type of a closed expression. Just put the cursor nearby an expression and press SHIFT+TAB.
- You can ask Agda to compute the normal form of an expression enclosed in parantheses, such as `(λ x → x)` (already in normal form).

-->

In [1]:
module code.conn where -- from before

infixr 2 _∧_
data _∧_ (A : Set) (B : Set) : Set where
  ⟨_,_⟩ : A → B → A ∧ B
  
fst : {A B : Set} → A ∧ B → A
fst ⟨ a , _ ⟩ = a

snd : {A B : Set} → A ∧ B → B
snd ⟨ _ , b ⟩ = b

infixr 1 _∨_ 
data _∨_ (A : Set) (B : Set) : Set where
    left : A → A ∨ B
    right : B → A ∨ B

data ⊤ : Set where
  tt : ⊤
  
data ⊥ : Set where

⊥-elim : {A : Set} → ⊥ → A
⊥-elim ()

infix 3 ¬_ -- higher priority than ∨ and ∧

¬_ : Set → Set
¬ A = A → ⊥

infixr 1 _⟺_
_⟺_ : (A : Set) (B : Set) → Set
A ⟺ B = (A → B) ∧ (B → A)

In [2]:
module code.nat where

data ℕ : Set where
  zero : ℕ
  suc : ℕ → ℕ

infixl 5 _+_
infixl 6 _*_ -- higher priority than _+_

-- left associative: a + b + c ≡ (a + b) + c

-- recursive function on an inductive type
_+_ : ℕ → ℕ → ℕ
zero + m = m
(suc n) + m = suc (n + m)

_*_ : ℕ → ℕ → ℕ
zero * _  =  zero
suc m * n  =  n + (m * n)

-- we can use standard numerals 0, 1, ...
-- as abbreviations for zero, suc zero, ...
{-# BUILTIN NATURAL ℕ #-}

In [5]:
module code.list where
open import code.conn public
open import code.nat public

infixr 5 _∷_
data _* (A : Set) : Set where
  ε : A *
  _∷_ : A → A * → A *

--length : {A : Set} → A * → ℕ
--length ε = 0
--length (_ ∷ xs) = suc (length xs)

infix 4 _≡L_
data _≡L_ {A : Set} : A * → A * → Set where
  ε≡ε : ε ≡L ε
  ∷≡∷ : {a : A} {as bs : A *} → as ≡L bs → a ∷ as ≡L a ∷ bs

≡L-refl : {A : Set} {as : A *} → as ≡L as
≡L-refl {A} {ε} = ε≡ε
≡L-refl {A} {a ∷ as} = ∷≡∷ (≡L-refl {A} {as})

≡L-sym : {A : Set} {as bs : A *} → as ≡L bs → bs ≡L as
≡L-sym ε≡ε = ε≡ε
≡L-sym {A} (∷≡∷ {a} {as} {bs} as≡bs) = ∷≡∷ {A} {a} {bs} {as} (≡L-sym {A} {as} {bs} as≡bs)

postulate ≡L-trans : {A : Set} {as bs cs : A *} → as ≡L bs → bs ≡L cs → as ≡L cs

# Decidables

````
module code.test where
````

# Vectors and internal verification

````
module code.vector where

open import code.nat
open import code.list -- hiding (_++_)

infixr 5 _∷_
infixr 5 _++_

-- indexed and parametrised type
-- the parameter is the type A and the index is the dimension n
data Vector (A : Set) : ℕ → Set where
  ε : Vector A zero
  _∷_ : {n : ℕ} → A → Vector A n → Vector A (suc n)

_++_ : {A : Set} {m n : ℕ} → Vector A m → Vector A n → Vector A (m + n)
ε ++ ys = ys
(x ∷ xs) ++ ys = x ∷ (xs ++ ys)

toList : {A : Set} {n : ℕ} → Vector A n → A *
toList ε = ε
toList (x ∷ xs) = x ∷ toList xs

-- more difficult
--fromList : {A : Set} → (xs : A *) → Vector A (length xs)
--fromList ε = ε
--fromList (x ∷ xs) = x ∷ fromList xs

map : {A B : Set} {n : ℕ} → (A → B) → Vector A n → Vector B n
map f ε = ε
map f (x ∷ xs) = f x ∷ map f xs

foldr : {A B : Set} {n : ℕ} → (A → B → B) → B → Vector A n → B
foldr _ e ε = e
foldr _⊗_ e (x ∷ xs) = x ⊗ foldr _⊗_ e xs

-- need to access the hidden parameter!
vlength : {A : Set} {n : ℕ} → Vector A n → ℕ
vlength {A} {n} _ = n
````

In [ ]:
module code.hello where

# Type-level programming